In [ ]:
!pip install pretty_midi
import sys
sys.path.insert(0,'../input/folkmusic')
import midi

In [ ]:
sample=midi.midi_datasets_to_samples(['../input/folkmusic/dataset'],48)

In [ ]:
print(sample.shape)
16*(sample.shape[0]//16)
sample=sample[0:16*(sample.shape[0]//16),:,:]
sample=sample.reshape(-1,16,96,96)
sample.shape


In [ ]:
import numpy as np
x_in=sample
y_shape=x_in.shape
y_train=x_in
x_train=y_train
test_ix = 0
y_test_song = np.copy(y_train[test_ix:test_ix+1])
x_test_song = np.copy(x_train[test_ix:test_ix+1])


In [ ]:
import sys, random, os
import numpy as np
from matplotlib import pyplot as plt
import keras
print("Keras Version: " + keras.__version__)
from keras.layers import Input, Dense, Activation, Dropout, Flatten, Reshape, Permute, RepeatVector, ActivityRegularization, TimeDistributed, Lambda, SpatialDropout1D
from keras.layers.convolutional import Conv1D, Conv2D, Conv2DTranspose, UpSampling2D, ZeroPadding2D
from keras.layers.embeddings import Embedding
from keras.layers.local import LocallyConnected2D
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.noise import GaussianNoise
from keras.layers.normalization import BatchNormalization
from keras.layers.recurrent import LSTM, SimpleRNN
from keras.initializers import RandomNormal
from keras.losses import binary_crossentropy
from keras.models import Model, Sequential, load_model
from keras.optimizers import Adam, RMSprop, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras.utils import plot_model
from keras import backend as K
from keras import regularizers
from keras.engine.topology import Layer
K.set_image_data_format('channels_first')
NUM_EPOCHS = 2000
LR = 0.001
CONTINUE_TRAIN = False
PLAY_ONLY = False
USE_EMBEDDING = False
USE_VAE = False
WRITE_HISTORY = True
NUM_RAND_SONGS = 10
DO_RATE = 0.1
BN_M = 0.9
VAE_B1 = 0.02
VAE_B2 = 0.1

BATCH_SIZE = 350
MAX_LENGTH = 16
PARAM_SIZE = 120
NUM_OFFSETS = 16 if USE_EMBEDDING else 1

In [ ]:
y_shape=(1632,16,96,96)
x_shape=(1632,16,96,96)
if CONTINUE_TRAIN or PLAY_ONLY:
	print("Loading Model...")
	model = load_model('model.h5', custom_objects=custom_objects)
else:
	print("Building Model...")

	if USE_EMBEDDING:
		x_in = Input(shape=x_shape[1:])
		x = Embedding(x_train.shape[0], PARAM_SIZE, input_length=1)(x_in)
		x = Flatten(name='pre_encoder')(x)
	else:
		x_in = Input(shape=y_shape[1:])
		x = Reshape((y_shape[1], -1))(x_in)
		print (K.int_shape(x))
		
		x = TimeDistributed(Dense(2000, activation='relu'))(x)
		print (K.int_shape(x))
		
		x = TimeDistributed(Dense(200, activation='relu'))(x)
		print (K.int_shape(x))

		x = Flatten()(x)
		print (K.int_shape(x))

		x = Dense(1600, activation='relu')(x)
		print (K.int_shape(x))
		
		if USE_VAE:
			z_mean = Dense(PARAM_SIZE)(x)
			z_log_sigma_sq = Dense(PARAM_SIZE)(x)
			x = Lambda(vae_sampling, output_shape=(PARAM_SIZE,), name='pre_encoder')([z_mean, z_log_sigma_sq])
		else:
			x = Dense(PARAM_SIZE)(x)
			x = BatchNormalization(momentum=BN_M, name='pre_encoder')(x)
	print (K.int_shape(x))
	
	x = Dense(1600, name='encoder')(x)
	x = BatchNormalization(momentum=BN_M)(x)
	x = Activation('relu')(x)
	if DO_RATE > 0:
		x = Dropout(DO_RATE)(x)
	print (K.int_shape(x))

	x = Dense(MAX_LENGTH * 200)(x)
	print (K.int_shape(x))
	x = Reshape((MAX_LENGTH, 200))(x)
	x = TimeDistributed(BatchNormalization(momentum=BN_M))(x)
	x = Activation('relu')(x)
	if DO_RATE > 0:
		x = Dropout(DO_RATE)(x)
	print (K.int_shape(x))

	x = TimeDistributed(Dense(2000))(x)
	x = TimeDistributed(BatchNormalization(momentum=BN_M))(x)
	x = Activation('relu')(x)
	if DO_RATE > 0:
		x = Dropout(DO_RATE)(x)
	print (K.int_shape(x))

	x = TimeDistributed(Dense(y_shape[2] * y_shape[3], activation='sigmoid'))(x)
	print (K.int_shape(x))
	x = Reshape((y_shape[1], y_shape[2], y_shape[3]))(x)
	print (K.int_shape(x))
	
	if USE_VAE:
		model = Model(x_in, x)
		model.compile(optimizer=Adam(lr=LR), loss=vae_loss)
	else:
		model = Model(x_in, x)
		model.compile(optimizer=RMSprop(lr=LR), loss='binary_crossentropy')

	plot_model(model, to_file='model.png', show_shapes=True)

In [ ]:
print ("Compiling SubModels...")
func = K.function([model.get_layer('encoder').input, K.learning_phase()],
				  [model.layers[-1].output])
enc = Model(inputs=model.input, outputs=model.get_layer('pre_encoder').output)

rand_vecs = np.random.normal(0.0, 1.0, (NUM_RAND_SONGS, PARAM_SIZE))
np.save('rand.npy', rand_vecs)


In [ ]:
print ("Training...")
#save_config()
train_loss = []
ofs = 0

for iter in range(NUM_EPOCHS):
	if USE_EMBEDDING:
		history = model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=1)
	else:
		history = model.fit(y_train, y_train, batch_size=BATCH_SIZE, epochs=1)

	loss = history.history["loss"][-1]
	train_loss.append(loss)
	print("Train Loss: " + str(train_loss[-1]))

	
	i = iter + 1
	if i in [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 120, 140, 160, 180, 200, 250, 300, 350, 400, 450] or (i % 100 == 0):
		write_dir = ''
		model.save('model.h5')
		print ("Saved")

		if USE_EMBEDDING:
			y_song = model.predict(x_test_song, batch_size=BATCH_SIZE)[0]
		else:
            
			y_song = model.predict(y_test_song, batch_size=BATCH_SIZE)[0]
			song=y_song.reshape(-1,96)
			song=song.T
			a=np.zeros((128,song.shape[1]))
			a[16:112,:]=song
			midi.piano_roll_to_midi(a,0,fs=48)

		#util.samples_to_pics(write_dir + 'test', y_song)


		#make_rand_songs_normalized(write_dir, rand_vecs)
            
print ("Done")



In [ ]:
song=y_song.reshape(-1,96)

In [ ]:
a=np.zeros((128,song.shape[1]))

In [ ]:
a[16:112,:]=song